In [25]:
import math

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns 
sns.set()
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import (
    Dense,
    Input,
    Bidirectional,
    Dropout,
    LSTM,
)
import keras.backend as K

In [6]:
df = pd.read_csv('health.csv')
values = df['blood sugar'].values

In [26]:
prediction_window  = 3
seed_window = 10
X_train = []
y_train = []
X_test = []
y_test = []

train_size = math.ceil(len(values) * .66)
test_size = len(values) - train_size
for i in range(train_size - (prediction_window + seed_window)):
    X_train.append(values[i:i + seed_window])
    y_train.append(values[i + seed_window:i + seed_window + prediction_window])

for i in range(train_size+1,len(values) - (prediction_window + seed_window) ):
    X_test.append(values[i:i + seed_window])
    y_test.append(values[i + seed_window:i + seed_window + prediction_window])

X_train = np.row_stack(X_train)
y_train = np.row_stack(y_train)
X_train = X_train[:, :, np.newaxis]
X_test = np.row_stack(X_test)
y_test = np.row_stack(y_test)
X_test = X_test[:, :, np.newaxis]

In [27]:
y_train.shape

(14884, 3)

In [28]:
X_train.shape

(14884, 10, 1)

In [29]:
X_test.shape

(7660, 10, 1)

In [30]:
y_test.shape

(7660, 3)

In [31]:
values.shape

(22571,)

In [23]:
df.head()

,Unnamed: 0,Steps,Flights,Finish,Hour,date,blood sugar,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9
0,0,0.0,0.0,2018-03-07 00:00:00,2018-03-07 00:00:00,2018-03-07 00:00:21,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.0,0.0,2018-03-07 00:00:00,2018-03-07 00:00:00,2018-03-07 00:05:20,101.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.0,0.0,2018-03-07 00:00:00,2018-03-07 00:00:00,2018-03-07 00:10:20,98.0,101.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.0,0.0,2018-03-07 00:00:00,2018-03-07 00:00:00,2018-03-07 00:15:20,95.0,98.0,101.0,104.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.0,0.0,2018-03-07 00:00:00,2018-03-07 00:00:00,2018-03-07 00:20:19,91.0,95.0,98.0,101.0,104.0,NaN,NaN,NaN,NaN,NaN


In [32]:
K.clear_session()
# You'll probably need to increase recur_size 
# with more complex curves or larger windows.
recur_size = 64
feature_vector_size = 1

inputs = Input(shape=(None, feature_vector_size))
x = (LSTM(recur_size))(inputs)
outputs = Dense(prediction_window, activation='linear')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='mse', optimizer='rmsprop', metrics=['mse', 'mae', 'mape'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 1)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 17,091
Trainable params: 17,091
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X_train, y_train, epochs=50, batch_size=128, validation_data=(X_test, y_test))

Train on 14884 samples, validate on 7660 samples
Epoch 1/50
14884/14884 [==============================] - 3s 235us/step - loss: 25104.5890 - mean_squared_error: 25104.5890 - mean_absolute_error: 146.1475 - mean_absolute_percentage_error: 97.6925 - val_loss: 28214.0489 - val_mean_squared_error: 28214.0489 - val_mean_absolute_error: 153.7109 - val_mean_absolute_percentage_error: 96.4263
Epoch 2/50
14884/14884 [==============================] - 3s 196us/step - loss: 24202.7248 - mean_squared_error: 24202.7248 - mean_absolute_error: 143.0249 - mean_absolute_percentage_error: 95.1470 - val_loss: 27374.3290 - val_mean_squared_error: 27374.3290 - val_mean_absolute_error: 150.9489 - val_mean_absolute_percentage_error: 94.2736
Epoch 3/50
14884/14884 [==============================] - 3s 203us/step - loss: 23416.0909 - mean_squared_error: 23416.0909 - mean_absolute_error: 140.2439 - mean_absolute_percentage_error: 92.8864 - val_loss: 26540.6799 - val_mean_squared_error: 26540.6799 - val_mean_ab

14884/14884 [==============================] - 3s 215us/step - loss: 6691.0070 - mean_squared_error: 6691.0070 - mean_absolute_error: 61.5715 - mean_absolute_percentage_error: 34.2331 - val_loss: 8395.3603 - val_mean_squared_error: 8395.3603 - val_mean_absolute_error: 68.8555 - val_mean_absolute_percentage_error: 35.5512
Epoch 26/50
14884/14884 [==============================] - 3s 203us/step - loss: 6211.3712 - mean_squared_error: 6211.3712 - mean_absolute_error: 58.3691 - mean_absolute_percentage_error: 32.0365 - val_loss: 7861.7012 - val_mean_squared_error: 7861.7012 - val_mean_absolute_error: 65.8057 - val_mean_absolute_percentage_error: 33.7366
Epoch 27/50
14884/14884 [==============================] - 3s 204us/step - loss: 5761.7307 - mean_squared_error: 5761.7307 - mean_absolute_error: 55.3015 - mean_absolute_percentage_error: 30.0273 - val_loss: 7351.8653 - val_mean_squared_error: 7351.8653 - val_mean_absolute_error: 62.8236 - val_mean_absolute_percentage_error: 32.0105
Epoch 2

14884/14884 [==============================] - 3s 206us/step - loss: 770.9774 - mean_squared_error: 770.9774 - mean_absolute_error: 15.2182 - mean_absolute_percentage_error: 8.8272 - val_loss: 1250.9424 - val_mean_squared_error: 1250.9424 - val_mean_absolute_error: 20.5926 - val_mean_absolute_percentage_error: 11.4804
Epoch 50/50
14884/14884 [==============================] - 3s 208us/step - loss: 696.5423 - mean_squared_error: 696.5423 - mean_absolute_error: 14.3106 - mean_absolute_percentage_error: 8.3969 - val_loss: 1152.1417 - val_mean_squared_error: 1152.1417 - val_mean_absolute_error: 19.4655 - val_mean_absolute_percentage_error: 10.5231


In [60]:
test_arr = np.array([150,155,156,165,170,400,178,100000000,600,40]).transpose()
test_arr = test_arr[np.newaxis,:,np.newaxis]
model.predict(test_arr)

array([[117.754906, 118.755005, 119.94092 ]], dtype=float32)